In [22]:
import re
import json
import requests
import pandas as pd

from pprint import pprint

from src.club import dir_item, dir_proj, dir_tableau
import src.mongos as mg

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 'process_club()'

In [4]:
def process_club(club):
    category_list = [
        "['술집', '유흥주점']",
        "['술집', '맥주,호프']",
        "['술집', '바(BAR)']",
        "['스포츠,오락', '클럽']",
    ]

    club['category_str'] = club['category'].apply(lambda x: str(x))

    idx_club = club['category_str'].isin(category_list)
    club = club.loc[idx_club]
    
    club['abbrAddress'].fillna('', inplace=True)

    club['roadAddress_short'] = club['roadAddress'].apply(lambda x: re.search(r'.+[로길]\s[\d-]+', x).group(0).strip() if re.search(r'.+[로길]\s[\d-]+', x) else x)
    club['abbrAddress_short'] = club['abbrAddress'].apply(lambda x: re.search(r'.+[동]\s[\d-]+', x).group(0).strip() if re.search(r'.+[동]\s[\d-]+', x) else x)
    return club

# 'process_localdata()'

In [6]:
def process_localdata(df):
    df['도로명전체주소'].fillna('', inplace=True)
    df['소재지전체주소'].fillna('', inplace=True)
    return df

# Load 'cafeteria'

In [7]:
cafeteria = mg.read_mongo('club', 'cafeteria')
cafeteria = process_localdata(cafeteria)
cafeteria.head()

,개방서비스ID,개방서비스명,개방자치단체코드,건물소유구분명,공장사무직종업원수,공장생산직종업원수,공장판매직종업원수,관리번호,급수시설구분명,남성종사자수,다중이용업소여부,데이터갱신구분,데이터갱신일자,도로명우편번호,도로명전체주소,등급구분명,번호,보증액,본사종업원수,사업장명,상세영업상태명,상세영업상태코드,소재지면적,소재지우편번호,소재지전체주소,소재지전화,시설총규모,업태구분명,여성종사자수,영업상태구분코드,영업상태명,영업장주변구분명,월세액,위생업태명,인허가일자,인허가취소일자,자치구,재개업일자,전통업소주된음식,전통업소지정번호,좌표정보(X),좌표정보(Y),총종업원수,최종수정시점,폐업일자,홈페이지,휴업시작일자,휴업종료일자
0,07_24_04_P,일반음식점,3020000,NaN,NaN,NaN,NaN,3020000-101-2010-00238,NaN,NaN,N,I,2018-08-31 23:59:59.0,4350.0,"서울특별시 용산구 이태원로 179 (이태원동,외(119-25) 해밀턴쇼핑센터 22,23호(지상2층))",NaN,36102,NaN,NaN,레이지캣,영업,1,171.00,140863.0,"서울특별시 용산구 이태원동 108-9번지 외(119-25) 해밀턴쇼핑센터 22,23호(지상2층)",02 7491390,171.00,경양식,NaN,1,영업/정상,NaN,NaN,경양식,20101028,NaN,용산구,NaN,NaN,NaN,199387.896367,448055.048870,NaN,20180122110632,NaN,NaN,NaN,NaN
1,07_24_04_P,일반음식점,3020000,NaN,NaN,NaN,NaN,3020000-101-1997-03975,상수도전용,0.0,N,I,2018-08-31 23:59:59.0,4405.0,"서울특별시 용산구 우사단로 42 (이태원동, 136-45 지상1층)",지도,36103,NaN,NaN,텍사스3,영업,1,34.24,140858.0,서울특별시 용산구 이태원동 136-45번지 지상1층,02 7967676,34.24,분식,0.0,1,영업/정상,유흥업소밀집지역,NaN,분식,19970502,NaN,용산구,NaN,NaN,NaN,199557.034319,447959.847009,NaN,20161010144738,NaN,NaN,NaN,NaN
2,07_24_04_P,일반음식점,3020000,NaN,NaN,NaN,NaN,3020000-101-1990-06081,상수도전용,1.0,N,I,2018-08-31 23:59:59.0,4382.0,"서울특별시 용산구 한강대로52길 17-14 (한강로2가,(지상1층))",지도,36104,NaN,NaN,풍년삼계탕,영업,1,31.28,140872.0,서울특별시 용산구 한강로2가 27-3번지 (지상1층),0207933556,31.28,한식,1.0,1,영업/정상,기타,NaN,한식,19900403,NaN,용산구,NaN,NaN,NaN,197510.132078,447798.512605,NaN,20110222154645,NaN,NaN,NaN,NaN
3,07_24_04_P,일반음식점,3020000,NaN,NaN,NaN,NaN,3020000-101-1990-04715,상수도전용,0.0,N,I,2018-08-31 23:59:59.0,4324.0,"서울특별시 용산구 한강대로104길 53 (동자동,(지상1층))",기타,36105,NaN,NaN,처갓집양념치킨,영업,1,32.92,140820.0,서울특별시 용산구 동자동 19-33번지 (지상1층),02 7559595,32.92,분식,0.0,1,영업/정상,기타,NaN,분식,19901215,NaN,용산구,NaN,NaN,NaN,197742.265234,449810.270608,NaN,20090219154943,NaN,NaN,NaN,NaN
4,07_24_04_P,일반음식점,3020000,NaN,NaN,NaN,NaN,3020000-101-1991-01272,상수도전용,2.0,N,I,2018-08-31 23:59:59.0,4373.0,"서울특별시 용산구 청파로 48 (한강로3가, 16-58 지하1층)",을,36106,NaN,NaN,연경뷔페,영업,1,169.22,140879.0,서울특별시 용산구 한강로3가 16-58번지 지하1층,0207014773,169.22,경양식,2.0,1,영업/정상,기타,NaN,경양식,19910422,NaN,용산구,NaN,NaN,NaN,196118.749576,447760.157527,NaN,20130819115431,NaN,NaN,NaN,NaN


In [12]:
cafeteria.columns

Index(['개방서비스ID', '개방서비스명', '개방자치단체코드', '건물소유구분명', '공장사무직종업원수', '공장생산직종업원수',
       '공장판매직종업원수', '관리번호', '급수시설구분명', '남성종사자수', '다중이용업소여부', '데이터갱신구분',
       '데이터갱신일자', '도로명우편번호', '도로명전체주소', '등급구분명', '번호', '보증액', '본사종업원수', '사업장명',
       '상세영업상태명', '상세영업상태코드', '소재지면적', '소재지우편번호', '소재지전체주소', '소재지전화', '시설총규모',
       '업태구분명', '여성종사자수', '영업상태구분코드', '영업상태명', '영업장주변구분명', '월세액', '위생업태명',
       '인허가일자', '인허가취소일자', '자치구', '재개업일자', '전통업소주된음식', '전통업소지정번호', '좌표정보(X)',
       '좌표정보(Y)', '총종업원수', '최종수정시점', '폐업일자', '홈페이지', '휴업시작일자', '휴업종료일자'],
      dtype='object')

# Columns

In [17]:
cols = ['사업장명', '인허가일자', '도로명전체주소', '소재지면적']

# Index

In [23]:
idx = [
    19564, 16772, 21776, 19904, 22005, 22746, 18642, 19920, 18745, 15561, 962, 1136, 1537, 3632, 224, 991, 324, 1533, 4151, 1796, 3284, 4319, 705, 3203, 518, 2232, 2904, 2888, 2452, 4288, 1731, 826, 801, 6998, 9679, 5726, 9764, 11124, 6633, 10130, 7356, 8804, 5039, 9078, 8581, 7957, 8388, 5819, 12084, 7126, 7017, 8856, 11273, 7556, 5375, 6185, 11337, 7841, 5178, 11933,
]
result = cafeteria.loc[cafeteria.index.isin(idx), cols].sort_values('인허가일자', ascending=True)

table = {
    '용산구': '이태원',
    '마포구': '홍대',
    '강남구': '강남',
}

def get_region(x):
    for k, v in table.items():
        if k in x:
            return v

result['지역'] = result['도로명전체주소'].apply(get_region)
result = result.loc[result['지역'].isin(['이태원', '강남'])]
result

,사업장명,인허가일자,도로명전체주소,소재지면적,지역
3632,바스타드,19830928,"서울특별시 용산구 우사단로14길 3 (이태원동,(지상3층))",152.29,이태원
4151,졸리조커,19851005,"서울특별시 용산구 이태원로27가길 36 (이태원동, 지하1층,지상1층)",182.68,이태원
2232,주식회사오퓸,19930918,"서울특별시 용산구 이태원로 189 (이태원동, 123-33 지하1층)",248.40,이태원
18745,제이,19931227,"서울특별시 강남구 도산대로94길 20, 지하2층 (청담동)",103.95,강남
4319,미드나잇(MIDNIGHT),19941110,"서울특별시 용산구 우사단로 46 (이태원동,(지하1층))",262.75,이태원
3203,헬리오스,19990611,"서울특별시 용산구 이태원로 195 (이태원동,(지상2층))",227.88,이태원
18642,핑가스존,20000720,"서울특별시 강남구 도산대로53길 32 (신사동,(지상2층))",91.11,강남
2888,아울컴퍼니,20040730,"서울특별시 용산구 이태원로27가길 9 (이태원동,(지상2층))",88.32,이태원
2452,무브,20070509,"서울특별시 용산구 보광로60길 22 (이태원동,(지하2층))",222.00,이태원
324,더소울바이제스(THE SOUL BY JESS),20071218,"서울특별시 용산구 이태원로27가길 5, 지하1층 (이태원동)",122.45,이태원


In [24]:
# result.to_csv(dir_tableau + 'allowed_date.tsv', sep='\t', index=False)

# Load 'liquor'

In [8]:
liquor = mg.read_mongo('club', 'liquor')
liquor = process_localdata(liquor)
liquor.head()

,개방서비스ID,개방서비스명,개방자치단체코드,건물소유구분명,공장사무직종업원수,공장생산직종업원수,공장판매직종업원수,관리번호,급수시설구분명,남성종사자수,다중이용업소여부,데이터갱신구분,데이터갱신일자,도로명우편번호,도로명전체주소,등급구분명,번호,보증액,본사종업원수,사업장명,상세영업상태명,상세영업상태코드,소재지면적,소재지우편번호,소재지전체주소,소재지전화,시설총규모,업태구분명,여성종사자수,영업상태구분코드,영업상태명,영업장주변구분명,월세액,위생업태명,인허가일자,인허가취소일자,자치구,재개업일자,전통업소주된음식,전통업소지정번호,좌표정보(X),좌표정보(Y),총종업원수,최종수정시점,폐업일자,홈페이지,휴업시작일자,휴업종료일자
0,07_23_02_P,유흥주점영업,3000000,NaN,NaN,NaN,NaN,3000000-102-1974-08292,상수도전용,0.0,N,U,2018-11-29 02:40:00.0,3149.0,"서울특별시 종로구 인사동5길 42 (관훈동,지하1층)",갑,5,NaN,NaN,엑스포,영업,1,NaN,110300.0,서울특별시 종로구 관훈동 198-8번지 지하1층,02 7303697,121.93,룸살롱,0.0,1,영업/정상,기타,NaN,룸살롱,19740510,NaN,종로구,NaN,NaN,NaN,198500.447559,452243.433655,NaN,20181127173604,NaN,NaN,NaN,NaN
1,07_23_02_P,유흥주점영업,3000000,NaN,NaN,NaN,NaN,3000000-102-1985-08261,상수도전용,2.0,N,I,2018-08-31 23:59:59.0,NaN,,갑,8,NaN,NaN,허리우드,영업,1,NaN,110320.0,서울특별시 종로구 낙원동 93번지 (지하1층),02 7423076,112.60,스텐드바,3.0,1,영업/정상,기타,NaN,스텐드바,19850122,NaN,종로구,NaN,NaN,NaN,198921.316373,452310.256279,NaN,20130107152739,NaN,NaN,NaN,NaN
2,07_23_02_P,유흥주점영업,3000000,NaN,NaN,NaN,NaN,3000000-102-1984-08322,상수도전용,2.0,N,I,2018-08-31 23:59:59.0,3192.0,서울특별시 종로구 수표로18길 6-8 (관수동),갑,10,NaN,NaN,레드폭스비지니스클럽,영업,1,NaN,110420.0,서울특별시 종로구 관수동 143-0번지,0222792880,56.00,카바레,3.0,1,영업/정상,기타,NaN,카바레,19840921,NaN,종로구,NaN,NaN,NaN,199038.463024,451853.037310,NaN,20021220000000,NaN,NaN,NaN,NaN
3,07_23_02_P,유흥주점영업,3000000,NaN,NaN,NaN,NaN,3000000-102-1984-11439,상수도전용,0.0,N,I,2018-08-31 23:59:59.0,3139.0,"서울특별시 종로구 수표로22길 2 (낙원동,(지하1층))",갑,11,NaN,NaN,발리,영업,1,NaN,110320.0,서울특별시 종로구 낙원동 179번지 (지하1층),NaN,67.75,스텐드바,0.0,1,영업/정상,기타,NaN,스텐드바,19840908,NaN,종로구,NaN,NaN,NaN,199006.837233,452085.964431,NaN,20170110170940,NaN,NaN,NaN,NaN
4,07_23_02_P,유흥주점영업,3000000,NaN,NaN,NaN,NaN,3000000-102-1984-08344,NaN,NaN,N,U,2018-09-17 23:59:59.0,3149.0,"서울특별시 종로구 인사동9길 31, 지하1층 (견지동)",NaN,15,NaN,NaN,레드카펫,영업,1,NaN,110170.0,서울특별시 종로구 견지동 92-1번지 지하1층,NaN,69.60,요정,NaN,1,영업/정상,NaN,NaN,요정,19840623,NaN,종로구,NaN,NaN,NaN,198477.624935,452292.215684,NaN,20180917151450,NaN,NaN,NaN,NaN
